# Prompt Engineering with Weave

This notebook demonstrates the prompt engineering capabilities of the Weave framework.

In [ ]:
from weave.prompts import PromptTemplate, PromptLibrary, PromptOptimizer
from weave.llms import OpenAILLM

# Initialize LLM
llm = OpenAILLM(
    model="gpt-4o-mini",
    api_key="your-api-key"  # or use environment variable
)

## 1. Working with Prompt Templates

Create and use dynamic prompt templates.

In [ ]:
# Create a template
template = PromptTemplate(
    "Classify the sentiment of this $language text: $text",
    metadata={"task": "sentiment_analysis", "version": "1.0"}
)

# Render template
prompt = template.render({
    "language": "English",
    "text": "This product is amazing!"
})

print(f"Rendered prompt: {prompt}")
print(f"Template metadata: {template.get_metadata()}")

## 2. Using the Prompt Library

Work with pre-defined prompt templates.

In [ ]:
# Create library
library = PromptLibrary()

# Add custom template
qa_template = PromptTemplate(
    "Answer the question based on the context:\n\n"
    "Context: $context\n\n"
    "Question: $question",
    metadata={"task": "question_answering"}
)

library.add_template("qa", qa_template, category="specialized")

# List available templates
print("Available categories:")
for category in library.get_categories():
    templates = library.get_templates_in_category(category)
    print(f"{category}: {templates}")

# Use a template
template = library.get_template("qa")
prompt = template.render({
    "context": "The cat sat on the mat.",
    "question": "Where did the cat sit?"
})

print(f"\nRendered QA prompt: {prompt}")

## 3. Prompt Optimization

Optimize prompts using feedback loops.

In [ ]:
# Create optimizer
optimizer = PromptOptimizer(
    model_connector=llm,
    optimization_config={
        "max_tokens": 150,
        "temperature": 0.7
    }
)

# Define test cases
test_cases = [
    {
        "input": {
            "text": "This movie was fantastic!",
            "categories": "positive, negative, neutral"
        },
        "expected": "positive"
    },
    {
        "input": {
            "text": "I didn't enjoy this book at all.",
            "categories": "positive, negative, neutral"
        },
        "expected": "negative"
    }
]

# Define evaluation function
def evaluate_classification(response: str, expected: str) -> float:
    return 1.0 if response.strip().lower() == expected.lower() else 0.0

# Get template from library
template = library.get_template("classification")

# Optimize template
optimized = optimizer.optimize(
    template=template,
    test_cases=test_cases,
    evaluation_fn=evaluate_classification,
    num_iterations=3
)

print("Optimization history:")
for record in optimizer.get_optimization_history():
    print(f"Iteration {record['iteration']}: {record['score']}")

print(f"\nBest template:\n{optimized.template.template}")

## 4. Multi-Turn Prompts

Create conversation-style prompts.

In [ ]:
# Create conversation template
conversation_template = PromptTemplate(
    """Previous conversation:
$history

User: $user_input
Assistant: Let me help you with that."""
)

# Build conversation history
history = [
    {"role": "user", "content": "Hello!"},
    {"role": "assistant", "content": "Hi! How can I help you today?"},
    {"role": "user", "content": "I need help with my code."}
]

# Format history
formatted_history = "\n".join(
    f"{turn['role'].title()}: {turn['content']}"
    for turn in history
)

# Render template
prompt = conversation_template.render({
    "history": formatted_history,
    "user_input": "Can you explain how to use functions?"
})

print("Multi-turn prompt:")
print(prompt)

## 5. Saving and Loading Templates

Persist templates and optimization results.

In [ ]:
import tempfile
from pathlib import Path

# Save template to file
with tempfile.TemporaryDirectory() as temp_dir:
    template_path = Path(temp_dir) / "template.json"
    template.to_file(template_path)
    
    # Load template from file
    loaded_template = PromptTemplate.from_file(template_path)
    print(f"Loaded template: {loaded_template.template.template}")
    
    # Save entire library
    library_dir = Path(temp_dir) / "templates"
    library.save_to_directory(library_dir)
    
    # Load library from directory
    loaded_library = PromptLibrary.from_directory(library_dir)
    print(f"\nLoaded library categories: {loaded_library.get_categories()}")
    
    # Save optimization history
    history_path = Path(temp_dir) / "optimization_history.json"
    optimizer.save_history(history_path)
    
    # Load optimization history
    new_optimizer = PromptOptimizer(model_connector=llm)
    new_optimizer.load_history(history_path)
    print(f"\nLoaded optimization records: {len(new_optimizer.get_optimization_history())}")